In [2]:
#Google Drive mounted
from google.colab import drive
drive.mount("/content/Drive")

Mounted at /content/Drive


In [3]:
path1 = "/content/Drive/MyDrive/Fake News Detection/Fake.csv"
path2 = "/content/Drive/MyDrive/Fake News Detection/True.csv"

In [4]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords   #Remove words
from nltk.stem.porter import PorterStemmer  #Return original word
from sklearn.feature_extraction.text import TfidfVectorizer   #Feture extraction
from sklearn.model_selection import train_test_split  #Train and test
from sklearn.metrics import classification_report
from sklearn.linear_model import *   #As binary classification
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [5]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [7]:
df_fake = pd.read_csv(path1)
df_true = pd.read_csv(path2)

In [8]:
print(df_fake.shape)
print(df_true.shape)

(23481, 4)
(21417, 4)


In [9]:
df_fakes=df_fake.sample(n=500)

In [10]:
df_trues=df_true.sample(n=500)

In [11]:
df_fakes.shape

(500, 4)

In [12]:
df_trues.shape

(500, 4)

In [13]:
df_fakes.isnull().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [14]:
df_trues.isnull().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [15]:
df_fakes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 11533 to 2403
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    500 non-null    object
 1   text     500 non-null    object
 2   subject  500 non-null    object
 3   date     500 non-null    object
dtypes: object(4)
memory usage: 19.5+ KB


In [16]:
df_fakes.head()

,title,text,subject,date
11533,WATCH SOUR GRAPES NANCY PELOSI Refuse to Stand...,,politics,"Mar 1, 2017"
17974,LOL! New York Times Publishes Lengthy Article ...,While Facebook has hired an entire team of emp...,left-news,"Sep 14, 2017"
4807,There Was Something Weird About Trump’s Hair ...,"Boy, wasn t Wednesday an embarrassing day for ...",News,"September 1, 2016"
11312,THIS ISN’T OBAMA’S AMERICA ANYMORE! Women’s Ma...,Convicted Palestinian terrorist and leader o...,politics,"Mar 24, 2017"
13717,The “Brown” The Media Wont’ Cover Because He W...,If his name was Michael Brown if he robbed a l...,politics,"Jun 10, 2016"


In [17]:
df_trues.head()

,title,text,subject,date
10619,Clinton wins big in South Carolina on way to '...,"COLUMBIA, S.C. (Reuters) - U.S. Democratic pre...",politicsNews,"February 27, 2016"
4888,Trump vows appeal up to Supreme Court after lo...,HONOLULU/NEW YORK (Reuters) - A defiant Presid...,politicsNews,"March 16, 2017"
2612,Trump talks pardons amid probes of Russia role...,"NORFOLK, Va. (Reuters) - U.S. President Donald...",politicsNews,"July 22, 2017"
14791,China corruption could lead to Soviet-style co...,BEIJING (Reuters) - China must step up its bat...,worldnews,"November 15, 2017"
13492,"Passenger train derails in Spain, 21 hurt",MADRID (Reuters) - A passenger train derailed ...,worldnews,"November 29, 2017"


Fake-> 1
Real-> 0

In [18]:
df_fakes["target"]=1
df_trues["target"]=0

In [19]:
df_fakes.head()

,title,text,subject,date,target
11533,WATCH SOUR GRAPES NANCY PELOSI Refuse to Stand...,,politics,"Mar 1, 2017",1
17974,LOL! New York Times Publishes Lengthy Article ...,While Facebook has hired an entire team of emp...,left-news,"Sep 14, 2017",1
4807,There Was Something Weird About Trump’s Hair ...,"Boy, wasn t Wednesday an embarrassing day for ...",News,"September 1, 2016",1
11312,THIS ISN’T OBAMA’S AMERICA ANYMORE! Women’s Ma...,Convicted Palestinian terrorist and leader o...,politics,"Mar 24, 2017",1
13717,The “Brown” The Media Wont’ Cover Because He W...,If his name was Michael Brown if he robbed a l...,politics,"Jun 10, 2016",1


In [20]:
df_trues.head()

,title,text,subject,date,target
10619,Clinton wins big in South Carolina on way to '...,"COLUMBIA, S.C. (Reuters) - U.S. Democratic pre...",politicsNews,"February 27, 2016",0
4888,Trump vows appeal up to Supreme Court after lo...,HONOLULU/NEW YORK (Reuters) - A defiant Presid...,politicsNews,"March 16, 2017",0
2612,Trump talks pardons amid probes of Russia role...,"NORFOLK, Va. (Reuters) - U.S. President Donald...",politicsNews,"July 22, 2017",0
14791,China corruption could lead to Soviet-style co...,BEIJING (Reuters) - China must step up its bat...,worldnews,"November 15, 2017",0
13492,"Passenger train derails in Spain, 21 hurt",MADRID (Reuters) - A passenger train derailed ...,worldnews,"November 29, 2017",0


Concat two different dataset

In [21]:
df = pd.concat([df_trues,df_fakes],axis=0)

In [22]:
df.shape

(1000, 5)

In [23]:
df.head()

,title,text,subject,date,target
10619,Clinton wins big in South Carolina on way to '...,"COLUMBIA, S.C. (Reuters) - U.S. Democratic pre...",politicsNews,"February 27, 2016",0
4888,Trump vows appeal up to Supreme Court after lo...,HONOLULU/NEW YORK (Reuters) - A defiant Presid...,politicsNews,"March 16, 2017",0
2612,Trump talks pardons amid probes of Russia role...,"NORFOLK, Va. (Reuters) - U.S. President Donald...",politicsNews,"July 22, 2017",0
14791,China corruption could lead to Soviet-style co...,BEIJING (Reuters) - China must step up its bat...,worldnews,"November 15, 2017",0
13492,"Passenger train derails in Spain, 21 hurt",MADRID (Reuters) - A passenger train derailed ...,worldnews,"November 29, 2017",0


In [24]:
df.tail()

,title,text,subject,date,target
21777,(VIDEO) OBAMA ON LETTERMAN: “WE IGNORE POCKETS...,The stats don t lie on this one Obama s hoping...,left-news,"May 5, 2015",1
1721,The White House Just Tried To Pretend Trump W...,"Earlier today, Donald Trump whined about being...",News,"April 21, 2017",1
1813,New York Daily News Cover SAVAGES White House...,The New York Daily News is not known for subtl...,News,"April 12, 2017",1
17280,THE STATE THAT GETS MORE REFUGEES THAN ANY OTH...,Here s one sure way to turn a solidly red st...,Government News,"Jun 11, 2015",1
2403,Gabby Giffords Shames GOP Rep. For Using Her ...,Republicans across the country have been doing...,News,"February 23, 2017",1


title,text,subject,date-> one_column e convert

In [25]:
df["content"]=df["title"]+ ' ' +df['text']+ ' ' +df["subject"]+ ' ' +df["date"]

In [26]:
print(df["content"])

10619    Clinton wins big in South Carolina on way to '...
4888     Trump vows appeal up to Supreme Court after lo...
2612     Trump talks pardons amid probes of Russia role...
14791    China corruption could lead to Soviet-style co...
13492    Passenger train derails in Spain, 21 hurt MADR...
                               ...                        
21777    (VIDEO) OBAMA ON LETTERMAN: “WE IGNORE POCKETS...
1721      The White House Just Tried To Pretend Trump W...
1813      New York Daily News Cover SAVAGES White House...
17280    THE STATE THAT GETS MORE REFUGEES THAN ANY OTH...
2403      Gabby Giffords Shames GOP Rep. For Using Her ...
Name: content, Length: 1000, dtype: object


Stemming

In [27]:
port_stem=PorterStemmer()

In [28]:
def stemming(content):
  stem_content=re.sub("[^a-zA-Z]",' ',content)    #Remove regular Expression
  stem_content=stem_content.lower()   #Convert all as lower-case
  stem_content=stem_content.split()   #Split the word
  stem_content=[port_stem.stem(word) for word in stem_content if word not in stopwords.words('english')]
  stem_content=" ".join(stem_content)
  return stem_content

In [29]:
df["content"]=df["content"].apply(stemming)

# Sentiment Analysis

In [30]:
from textblob import TextBlob

In [31]:
def get_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity

In [32]:
df['sentiment'] = df['content'].apply(get_sentiment)

In [33]:

print(df["sentiment"])

10619    0.231996
4888     0.096980
2612     0.172956
14791   -0.107484
13492   -0.185476
           ...   
21777   -0.017497
1721     0.116667
1813     0.105962
17280    0.093002
2403     0.015289
Name: sentiment, Length: 1000, dtype: float64


In [34]:
df.head()

,title,text,subject,date,target,content,sentiment
10619,Clinton wins big in South Carolina on way to '...,"COLUMBIA, S.C. (Reuters) - U.S. Democratic pre...",politicsNews,"February 27, 2016",0,clinton win big south carolina way super tuesd...,0.231996
4888,Trump vows appeal up to Supreme Court after lo...,HONOLULU/NEW YORK (Reuters) - A defiant Presid...,politicsNews,"March 16, 2017",0,trump vow appeal suprem court loss travel ban ...,0.096980
2612,Trump talks pardons amid probes of Russia role...,"NORFOLK, Va. (Reuters) - U.S. President Donald...",politicsNews,"July 22, 2017",0,trump talk pardon amid probe russia role u ele...,0.172956
14791,China corruption could lead to Soviet-style co...,BEIJING (Reuters) - China must step up its bat...,worldnews,"November 15, 2017",0,china corrupt could lead soviet style collaps ...,-0.107484
13492,"Passenger train derails in Spain, 21 hurt",MADRID (Reuters) - A passenger train derailed ...,worldnews,"November 29, 2017",0,passeng train derail spain hurt madrid reuter ...,-0.185476


# Feature extraction using TfidfVectorizer

In [35]:
X=df[["content","sentiment"]].values
Y=df["target"].values

In [36]:
print(X)

[['clinton win big south carolina way super tuesday columbia c reuter u democrat presidenti candid hillari clinton crush rival berni sander south carolina primari saturday propel next week crucial super tuesday vote state wave momentum rout sander solidifi clinton statu strong front runner captur parti nomin nov elect quest becom america first woman presid nearli half vote count south carolina clinton led sander point margin dramat revers point loss state presid barack obama bitter primari battl former secretari state victori decis establish strength among black voter crucial democrat constitu make half parti primari elector south carolina win clinton appear look ahead gener elect matchup republican front runner donald trump billionair whose campaign slogan make america great call build wall border mexico despit hear need make america great america never stop great told cheer support columbia win instead build wall need tear barrier result clinton third victori first four democrat cont

In [37]:
tfidf_vectorizer=TfidfVectorizer()

In [38]:
tfidf_vectorizer.fit(X[: , 0])
X_text=tfidf_vectorizer.transform(X[: , 0])

In [39]:
print(X)

[['clinton win big south carolina way super tuesday columbia c reuter u democrat presidenti candid hillari clinton crush rival berni sander south carolina primari saturday propel next week crucial super tuesday vote state wave momentum rout sander solidifi clinton statu strong front runner captur parti nomin nov elect quest becom america first woman presid nearli half vote count south carolina clinton led sander point margin dramat revers point loss state presid barack obama bitter primari battl former secretari state victori decis establish strength among black voter crucial democrat constitu make half parti primari elector south carolina win clinton appear look ahead gener elect matchup republican front runner donald trump billionair whose campaign slogan make america great call build wall border mexico despit hear need make america great america never stop great told cheer support columbia win instead build wall need tear barrier result clinton third victori first four democrat cont

In [40]:
import numpy as np

In [41]:
X_with_sentiment = np.hstack((X_text.toarray(), X[:, 1].reshape(-1, 1)))

# Split(Train-Test)

In [42]:
X_train,X_test,Y_train,Y_test=train_test_split(X_with_sentiment,Y,test_size=0.2, random_state=2)

# Logistic Regression

In [43]:
model_lgr=LogisticRegression()

In [44]:
model_lgr.fit(X_train,Y_train)
predict_LGR=model_lgr.predict(X_train)
accuracy_train_LGR=accuracy_score(predict_LGR,Y_train)

In [45]:
print("Accuracy of Logistic Regression:",accuracy_train_LGR)

Accuracy of Logistic Regression: 0.995


In [46]:
pred_lgr = model_lgr.predict(X_test)

In [47]:
print(classification_report(Y_test,pred_lgr))

              precision    recall  f1-score   support

           0       0.96      0.94      0.95        98
           1       0.94      0.96      0.95       102

    accuracy                           0.95       200
   macro avg       0.95      0.95      0.95       200
weighted avg       0.95      0.95      0.95       200



#Decision Tree

In [48]:
from sklearn.tree import DecisionTreeClassifier

In [49]:
dt=DecisionTreeClassifier()
dt.fit(X_train, Y_train)

DecisionTreeClassifier()

In [50]:
predict_dt=dt.predict(X_train)
accuracy_train_dt=accuracy_score(predict_dt,Y_train)
print("Accuracy for DecisionTree:",accuracy_train_dt)

Accuracy for DecisionTree: 1.0


In [51]:
pred_dt = dt.predict(X_test)

In [52]:
print(classification_report(Y_test,pred_dt))

              precision    recall  f1-score   support

           0       0.99      0.97      0.98        98
           1       0.97      0.99      0.98       102

    accuracy                           0.98       200
   macro avg       0.98      0.98      0.98       200
weighted avg       0.98      0.98      0.98       200



# Random Forest

In [53]:
from sklearn.ensemble import RandomForestClassifier

In [54]:
rf=RandomForestClassifier()

In [55]:
rf.fit(X_train,Y_train)
predict_rf=rf.predict(X_train)
accuracy_train_rf=accuracy_score(predict_rf,Y_train)

In [56]:
print("Accuracy of Random Forest:",accuracy_train_rf)

Accuracy of Random Forest: 1.0


In [57]:
pred_rf = rf.predict(X_test)

In [58]:
print(classification_report(Y_test,pred_rf))

              precision    recall  f1-score   support

           0       0.99      0.98      0.98        98
           1       0.98      0.99      0.99       102

    accuracy                           0.98       200
   macro avg       0.99      0.98      0.98       200
weighted avg       0.99      0.98      0.98       200



# Gradient Boosting

In [59]:
from sklearn.ensemble import GradientBoostingClassifier

In [60]:
gdb=GradientBoostingClassifier()

In [61]:
gdb.fit(X_train,Y_train)
predict_gdb=gdb.predict(X_train)
accuracy_train_gdb=accuracy_score(predict_gdb,Y_train)

In [62]:
print("Accuracy of Gradient Boosting:",accuracy_train_gdb)

Accuracy of Gradient Boosting: 1.0


In [63]:
pred_gdb = gdb.predict(X_test)

In [64]:
print(classification_report(Y_test,pred_gdb))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99        98
           1       1.00      0.99      1.00       102

    accuracy                           0.99       200
   macro avg       0.99      1.00      0.99       200
weighted avg       1.00      0.99      1.00       200



# Naive Bias

In [65]:
gnb = GaussianNB()

In [66]:
gnb.fit(X_train, Y_train)
predict_gnb=gnb.predict(X_train)
accuracy_train_gnb=accuracy_score(predict_gdb,Y_train)

In [67]:
print("Accuracy of Naive Bias:",accuracy_train_gnb)

Accuracy of Naive Bias: 1.0


In [68]:
pred_gnb = gnb.predict(X_test)

In [69]:
print(classification_report(Y_test,pred_gnb))

              precision    recall  f1-score   support

           0       0.79      0.70      0.75        98
           1       0.74      0.82      0.78       102

    accuracy                           0.77       200
   macro avg       0.77      0.76      0.76       200
weighted avg       0.77      0.77      0.76       200



#Prediction

In [70]:
prediction=pred_lgr[0]

if (prediction==0):
  print("The news is true")
else:
  print("The news is fake")

The news is true


In [71]:
print("Sentiment analysis value for the first news article:", df.iloc[0]['sentiment'])

Sentiment analysis value for the first news article: 0.231995980210266


In [72]:
prediction=pred_lgr[1]

if (prediction==0):
  print("The news is true")
else:
  print("The news is fake")

The news is fake


In [73]:
print("Sentiment analysis value for the first news article:", df.iloc[1]['sentiment'])

Sentiment analysis value for the first news article: 0.09697993447993451
